# Import the Required Libraries

In [68]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

!pip install geocoder

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium

print("Libraries imported.")

Solving environment: done

# All requested packages already installed.

Libraries imported.


# Create the Dataset

In [69]:
# send the GET request
data = requests.get('https://en.wikipedia.org/wiki/Neighborhoods_in_New_Orleans').text

In [70]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [71]:
# create three lists to store table data
NeighborhoodList = []
LongitudeList = []
LatitudeList = []

In [72]:
def remove_html_tags(text):
    """Remove html tags from a string"""
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

In [73]:
# append the data into the respective lists

for row in soup.find('table').find_all('tr'):
    
    cells = row.find_all('td')
    if(len(cells) > 0):
        NeighborhoodList.append(remove_html_tags(str(cells[0])))
        LongitudeList.append(remove_html_tags(str(cells[1])))
        LatitudeList.append(remove_html_tags(str(cells[2])))

In [74]:
# create a new DataFrame from the three lists
neworleans_df = pd.DataFrame({"Neighborhood": NeighborhoodList,
                              "Latitude": LatitudeList,
                              "Longitude": LongitudeList})
                             
neworleans_df.head()

,Neighborhood,Latitude,Longitude
0,U.S. NAVAL BASE\n,29.94608469\n,-90.02609253\n
1,ALGIERS POINT\n,29.95246187\n,-90.0516057\n
2,WHITNEY\n,29.94720026\n,-90.04235744\n
3,AUDUBON\n,29.93299437\n,-90.12145042\n
4,OLD AURORA\n,29.92444011\n,-90\n


In [75]:
neworleans_df['Neighborhood'] = neworleans_df['Neighborhood'].replace('\n','', regex=True)
neworleans_df['Longitude'] = neworleans_df['Longitude'].replace('\n','', regex=True)
neworleans_df['Latitude'] = neworleans_df['Latitude'].replace('\n','', regex=True)
neworleans_df.head()

,Neighborhood,Latitude,Longitude
0,U.S. NAVAL BASE,29.94608469,-90.02609253
1,ALGIERS POINT,29.95246187,-90.0516057
2,WHITNEY,29.94720026,-90.04235744
3,AUDUBON,29.93299437,-90.12145042
4,OLD AURORA,29.92444011,-90


In [76]:
neworleans_df['Latitude'] = neworleans_df['Latitude'].astype(float)
neworleans_df['Longitude'] = neworleans_df['Longitude'].astype(float)

In [77]:
address = 'New Orleans, LA'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New Orleans are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New Orleans are 29.9499323, -90.0701156.


# Create the Map of New Orleans

In [78]:
map_neworleans = folium.Map(location=[latitude, longitude], zoom_start=12, control_scale=True)

# add markers to map
for neighborhood, lat, lng in zip(neworleans_df['Neighborhood'], neworleans_df['Latitude'], neworleans_df['Longitude']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_neworleans)  
    
map_neworleans

In [84]:
CLIENT_ID = 'TOLOKUEUMQ1XHATYY1NGEBKJ25JKJXEYFUY01HXAJC5C2KC1' # your Foursquare ID
CLIENT_SECRET = '1NTDBUWXWJCXQ1SW4RFIFOTSQMXQCP5UBUPSSW0KCEIKEYST' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TOLOKUEUMQ1XHATYY1NGEBKJ25JKJXEYFUY01HXAJC5C2KC1
CLIENT_SECRET:1NTDBUWXWJCXQ1SW4RFIFOTSQMXQCP5UBUPSSW0KCEIKEYST


# Perform High-Level Exploration of the Data

In [85]:
#Let's examine the French Quarter in our dataframe.
neworleans_df.loc[15, 'Neighborhood']

'FRENCH QUARTER'

In [86]:
neighborhood_latitude = neworleans_df.loc[15, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neworleans_df.loc[15, 'Longitude'] # neighborhood longitude value

neighborhood_name = neworleans_df.loc[15, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of FRENCH QUARTER are 29.95883865, -90.06437302.


In [136]:
#Now, let's get the top 100 venues that are in French Quarter within a radius of 700 meters.

radius = 500
LIMIT = 150

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    longitude, 
    radius, 
    LIMIT)

url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=TOLOKUEUMQ1XHATYY1NGEBKJ25JKJXEYFUY01HXAJC5C2KC1&client_secret=1NTDBUWXWJCXQ1SW4RFIFOTSQMXQCP5UBUPSSW0KCEIKEYST&v=20180605&ll=29.95949861,-90.0701156&radius=500&limit=150'

In [137]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ed9cad79fcb92001bf2e6ff'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Treme - Lafitte',
  'headerFullLocation': 'Treme - Lafitte, New Orleans',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 25,
  'suggestedBounds': {'ne': {'lat': 29.963998614500007,
    'lng': -90.064931258734},
   'sw': {'lat': 29.954998605499995, 'lng': -90.07529994126598}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54dec661498edc3a983897c1',
       'name': 'The Black Penny',
       'location': {'address': '700 N Rampart St',
        'lat': 29.960486806150996,
        'lng': -90.06841071087526,
        'lab

In [138]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [139]:
#cleaning the response json and structuring it into pandas dataframe
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,The Black Penny,Bar,29.960487,-90.068411
1,Backatown Coffee Parlour,Coffee Shop,29.958285,-90.071674
2,Jewel Of The South,Cocktail Bar,29.958538,-90.069523
3,FreeWheelin' Bike Tours,Bike Rental / Bike Share,29.957256,-90.069913
4,Congo Square,Park,29.961154,-90.068402


In [140]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

25 venues were returned by Foursquare.


## Explore Neighborhoods in New Orleans

In [ ]:
#Let's create a function to repeat the same process to all the neighborhoods in New Orleans

In [141]:
address = 'New Orleans, LA'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New Orleans are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of New Orleans are 29.9499323, -90.0701156.


In [142]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [143]:
neworleans_venues = getNearbyVenues(names=neworleans_df['Neighborhood'],
                                   latitudes=neworleans_df['Latitude'],
                                   longitudes=neworleans_df['Longitude']
                                  )

U.S. NAVAL BASE
ALGIERS POINT
WHITNEY
AUDUBON
OLD AURORA
B. W. COOPER
BAYOU ST. JOHN
BEHRMAN
BLACK PEARL
BROADMOOR
MARLYVILLE - FONTAINEBLEAU
GERT TOWN
MID-CITY
ST. CLAUDE
CENTRAL BUSINESS DISTRICT
FRENCH QUARTER
CENTRAL CITY
LAKE CATHERINE
VILLAGE DE LEST
VIAVANT - VENETIAN ISLES
NEW AURORA - ENGLISH TURN
TALL TIMBERS - BRECHTEL
FISCHER DEV
McDONOGH
LOWER GARDEN DISTRICT
ST. THOMAS DEV
EAST RIVERSIDE
IRISH CHANNEL
TOURO
MILAN
UPTOWN
WEST RIVERSIDE
EAST CARROLLTON
FRERET
GARDEN DISTRICT
LEONIDAS
HOLLYGROVE
TULANE - GRAVIER
TREME - LAFITTE
SEVENTH WARD
MARIGNY
ST. ROCH
DIXON
LAKEWOOD
NAVARRE
CITY PARK
LAKEVIEW
WEST END
LAKESHORE - LAKE VISTA
FILMORE
ST. BERNARD AREA
DILLARD
ST.   ANTHONY
LAKE TERRACE &amp; OAKS
MILNEBURG
PONTCHARTRAIN PARK
GENTILLY WOODS
GENTILLY TERRACE
DESIRE AREA
FLORIDA AREA
FLORIDA DEV
LOWER NINTH WARD
BYWATER
HOLY CROSS
PINES VILLAGE
PLUM ORCHARD
READ BLVD WEST
READ BLVD EAST
WEST LAKE FOREST
LITTLE WOODS
FAIRGROUNDS
IBERVILLE


In [144]:
print(neworleans_venues.shape)
neworleans_venues.head()

(898, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,U.S. NAVAL BASE,29.946085,-90.026093,The Mighty Missisippi,29.949695,-90.023710,Boat or Ferry
1,ALGIERS POINT,29.952462,-90.051606,Tout de Suite Café,29.952121,-90.051090,Café
2,ALGIERS POINT,29.952462,-90.051606,Congregation Coffee Roasters,29.951918,-90.053395,Coffee Shop
3,ALGIERS POINT,29.952462,-90.051606,Old Point Bar,29.954940,-90.050226,Bar
4,ALGIERS POINT,29.952462,-90.051606,The Crown & Anchor,29.951416,-90.054220,Bar


In [145]:
# number of venues for each neighborhood
neworleans_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
ALGIERS POINT,23,23,23,23,23,23
AUDUBON,9,9,9,9,9,9
B. W. COOPER,4,4,4,4,4,4
BAYOU ST. JOHN,8,8,8,8,8,8
BLACK PEARL,5,5,5,5,5,5
BROADMOOR,5,5,5,5,5,5
BYWATER,15,15,15,15,15,15
CENTRAL BUSINESS DISTRICT,100,100,100,100,100,100
CENTRAL CITY,7,7,7,7,7,7


In [146]:
print('There are {} uniques categories.'.format(len(neworleans_venues['Venue Category'].unique())))

There are 200 uniques categories.


## Analyze Each Neighborhood

In [147]:
# Use one hot encoding to analyze each neighborhood.
neworleans_onehot = pd.get_dummies(neworleans_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
neworleans_onehot['Neighborhood'] = neworleans_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [neworleans_onehot.columns[-1]] + list(neworleans_onehot.columns[:-1])
neworleans_onehot = neworleans_onehot[fixed_columns]

neworleans_onehot.head()


,Yoga Studio,ATM,Accessories Store,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Bed & Breakfast,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Burger Joint,Burrito Place,Bus Station,Bus Stop,Café,Cajun / Creole Restaurant,Campground,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Theater,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Discount Store,Dive Bar,Donut Shop,Eastern European Restaurant,Ethiopian Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Frame Store,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Hardware Store,Historic Site,History Museum,Home Service,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Restaurant,Indie Theater,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Lake,Light Rail Station,Lounge,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motorcycle Shop,Museum,Music Store,Music Venue,Nail Salon,National Park,Neighborhood,New American Restaurant,Nightclub,Nightlife Spot,Optical Shop,Other Great Outdoors,Outdoors & Recreation,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pier,Pizza Place,Playground,Plaza,Pool Hall,Pub,Public Art,Racetrack,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sports Bar,Stables,Steakhouse,Street Art,Student Center,Sushi Restaurant,Taco Place,Tapas Restaurant,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Tour Provider,Tourist Information Center,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,U.S. NAVAL BASE,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,ALGIERS POINT,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,ALGIERS POINT,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [148]:
neworleans_onehot.shape

(898, 200)

In [149]:
#Next, group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
neworleans_grouped = neworleans_onehot.groupby('Neighborhood').mean().reset_index()
neworleans_grouped.head()

,Neighborhood,Yoga Studio,ATM,Accessories Store,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Bed & Breakfast,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Burger Joint,Burrito Place,Bus Station,Bus Stop,Café,Cajun / Creole Restaurant,Campground,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Theater,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Discount Store,Dive Bar,Donut Shop,Eastern European Restaurant,Ethiopian Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Frame Store,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Hardware Store,Historic Site,History Museum,Home Service,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Restaurant,Indie Theater,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Lake,Light Rail Station,Lounge,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motorcycle Shop,Museum,Music Store,Music Venue,Nail Salon,National Park,New American Restaurant,Nightclub,Nightlife Spot,Optical Shop,Other Great Outdoors,Outdoors & Recreation,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pier,Pizza Place,Playground,Plaza,Pool Hall,Pub,Public Art,Racetrack,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sports Bar,Stables,Steakhouse,Street Art,Student Center,Sushi Restaurant,Taco Place,Tapas Restaurant,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Tour Provider,Tourist Information Center,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store
0,ALGIERS POINT,0.0,0.0,0.0,0.0,0.0,0.0,0.043478,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.130435,0.0,0.0,0.0,0.043478,0.0,0.0,0.0,0.0,0.0,0.086957,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.043478,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.043478,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.043478,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.043478,0.0,0.043478,0.0,0.00,0.0,0.0,0.0,0.086957,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.043478,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.043478,0.0,0.0,0.043478,0.0,0.0,0.0,0.000,0.000000,0.0,0.043478,0.0,0.0,0.0,0.0,0.043478,0.000,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.086957,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.043478,0.0
1,AUDUBON,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.111111,0.0,0.111111,0.0,0.0,0.0,

In [150]:
#confirm the new size
neworleans_grouped.shape

(66, 200)

In [151]:
#Let's print each neighborhood along with the top 5 most common venues

num_top_venues = 5

for hood in neworleans_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = neworleans_grouped[neworleans_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----ALGIERS POINT----
            venue  freq
0             Bar  0.13
1   Boat or Ferry  0.09
2   Historic Site  0.09
3  Scenic Lookout  0.09
4       Gift Shop  0.04


----AUDUBON----
                   venue  freq
0           Concert Hall  0.11
1  Outdoors & Recreation  0.11
2  College Arts Building  0.11
3                   Park  0.11
4            Coffee Shop  0.11


----B. W. COOPER----
                  venue  freq
0        Breakfast Spot  0.25
1     Recreation Center  0.25
2          Home Service  0.25
3  Gym / Fitness Center  0.25
4              Pharmacy  0.00


----BAYOU ST. JOHN----
                  venue  freq
0     Convenience Store  0.12
1           Gas Station  0.12
2  Other Great Outdoors  0.12
3            Food Truck  0.12
4        Sandwich Place  0.12


----BLACK PEARL----
            venue  freq
0  Farmers Market   0.2
1      Food Truck   0.2
2           Plaza   0.2
3  Scenic Lookout   0.2
4   Grocery Store   0.2


----BROADMOOR----
               venue  freq
0  Recrea

            venue  freq
0   Women's Store   0.5
1         Theater   0.5
2        Pharmacy   0.0
3       Nightclub   0.0
4  Nightlife Spot   0.0


----OLD AURORA----
            venue  freq
0    Concert Hall  0.25
1  General Travel  0.25
2     Snack Place  0.25
3      Playground  0.25
4       Pet Store  0.00


----PINES VILLAGE----
                venue  freq
0      Nightlife Spot  0.33
1      Discount Store  0.33
2  Seafood Restaurant  0.33
3         Yoga Studio  0.00
4            Pharmacy  0.00


----PLUM ORCHARD----
                 venue  freq
0  American Restaurant  0.33
1  Fried Chicken Joint  0.33
2          Auto Garage  0.33
3          Yoga Studio  0.00
4                 Pier  0.00


----PONTCHARTRAIN PARK----
            venue  freq
0            Park   1.0
1     Yoga Studio   0.0
2   National Park   0.0
3       Nightclub   0.0
4  Nightlife Spot   0.0


----READ BLVD EAST----
              venue  freq
0  Insurance Office  0.25
1              Bank  0.25
2    Scenic Lookout  0.25


In [152]:
#Put the data in a dataframe...
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [153]:
#function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [154]:
#and create the dataframe
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = neworleans_grouped['Neighborhood']

for ind in np.arange(neworleans_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(neworleans_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ALGIERS POINT,Bar,Scenic Lookout,Historic Site,Boat or Ferry,Park,Gift Shop,Music Venue,Grocery Store,Coffee Shop,Bed & Breakfast
1,AUDUBON,Concert Hall,Park,College Arts Building,Coffee Shop,Plaza,Smoothie Shop,Sandwich Place,Outdoors & Recreation,Light Rail Station,Food
2,B. W. COOPER,Recreation Center,Breakfast Spot,Home Service,Gym / Fitness Center,Eastern European Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop
3,BAYOU ST. JOHN,Lounge,Convenience Store,Playground,Gas Station,Sandwich Place,Other Great Outdoors,Food Truck,Park,Women's Store,Food & Drink Shop
4,BLACK PEARL,Food Truck,Plaza,Farmers Market,Scenic Lookout,Grocery Store,Donut Shop,Food Court,Food & Drink Shop,Food,Flower Shop
5,BROADMOOR,Recreation Center,Food Truck,Coffee Shop,Taco Place,Music Venue,Ethiopian Restaurant,Frame Store,Food Court,Food & Drink Shop,Food
6,BYWATER,Boat or Ferry,Coffee Shop,New American Restaurant,Fried Chicken Joint,Seafood Restaurant,Caribbean Restaurant,Park,Discount Store,Gym / Fitness Center,Music Venue
7,CENTRAL BUSINESS DISTRICT,Hotel,Hotel Bar,Coffee Shop,Bar,Café,Restaurant,Sandwich Place,Pizza Place,Cajun / Creole Restaurant,Juice Bar
8,CENTRAL CITY,Park,Grocery Store,Fried Chicken Joint,Cosmetics Shop,Seafood Restaurant,Cajun / Creole Restaurant,Eastern European Restaurant,Food & Drink Shop,Food,Flower Shop
9,CITY PARK,Baseball Field,Park,Trail,Tennis Court,Women's Store,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop


In [155]:
# set number of clusters
kclusters = 4

neworleans_grouped_clustering = neworleans_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(neworleans_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [156]:
# add clustering labels

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

neworleans_merged = neworleans_df

# merge neworleans_grouped with neworleans_data to add latitude/longitude for each neighborhood
neworleans_merged = neworleans_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

#convert float to int in column 'Cluster Labels'
neworleans_merged[['Cluster Labels']] = neworleans_merged[['Cluster Labels']].astype('object')

neworleans_merged # check the last columns!

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,U.S. NAVAL BASE,29.946085,-90.026093,1,Boat or Ferry,Ethiopian Restaurant,French Restaurant,Frame Store,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop,Fast Food Restaurant
1,ALGIERS POINT,29.952462,-90.051606,1,Bar,Scenic Lookout,Historic Site,Boat or Ferry,Park,Gift Shop,Music Venue,Grocery Store,Coffee Shop,Bed & Breakfast
2,WHITNEY,29.947200,-90.042357,3,Nightclub,Women's Store,Ethiopian Restaurant,Frame Store,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop,Fast Food Restaurant
3,AUDUBON,29.932994,-90.121450,1,Concert Hall,Park,College Arts Building,Coffee Shop,Plaza,Smoothie Shop,Sandwich Place,Outdoors & Recreation,Light Rail Station,Food
4,OLD AURORA,29.924440,-90.000000,1,Concert Hall,Snack Place,Playground,General Travel,Women's Store,Ethiopian Restaurant,Food Court,Food & Drink Shop,Food,Flower Shop
5,B. W. COOPER,29.951774,-90.091753,1,Recreation Center,Breakfast Spot,Home Service,Gym / Fitness Center,Eastern European Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop
6,BAYOU ST. JOHN,29.976071,-90.086517,1,Lounge,Convenience Store,Playground,Gas Station,Sandwich Place,Other Great Outdoors,Food Truck,Park,Women's Store,Food & Drink Shop
7,BEHRMAN,29.934817,-90.026436,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,BLACK PEARL,29.935895,-90.134883,1,Food Truck,Plaza,Farmers Market,Scenic Lookout,Grocery Store,Donut Shop,Food Court,Food & Drink Shop,Food,Flower Shop
9,BROADMOOR,29.946568,-90.103812,1,Recreation Center,Food Truck,Coffee Shop,Taco Place,Music Venue,Ethiopian Restaurant,Frame Store,Food Court,Food & Drink Shop,Food


In [157]:
neworleans_merged=neworleans_merged.dropna()
#convert float to int in column 'Cluster Labels'
neworleans_merged[['Cluster Labels']] = neworleans_merged[['Cluster Labels']].astype('object')
neworleans_merged

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,U.S. NAVAL BASE,29.946085,-90.026093,1,Boat or Ferry,Ethiopian Restaurant,French Restaurant,Frame Store,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop,Fast Food Restaurant
1,ALGIERS POINT,29.952462,-90.051606,1,Bar,Scenic Lookout,Historic Site,Boat or Ferry,Park,Gift Shop,Music Venue,Grocery Store,Coffee Shop,Bed & Breakfast
2,WHITNEY,29.947200,-90.042357,3,Nightclub,Women's Store,Ethiopian Restaurant,Frame Store,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop,Fast Food Restaurant
3,AUDUBON,29.932994,-90.121450,1,Concert Hall,Park,College Arts Building,Coffee Shop,Plaza,Smoothie Shop,Sandwich Place,Outdoors & Recreation,Light Rail Station,Food
4,OLD AURORA,29.924440,-90.000000,1,Concert Hall,Snack Place,Playground,General Travel,Women's Store,Ethiopian Restaurant,Food Court,Food & Drink Shop,Food,Flower Shop
5,B. W. COOPER,29.951774,-90.091753,1,Recreation Center,Breakfast Spot,Home Service,Gym / Fitness Center,Eastern European Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop
6,BAYOU ST. JOHN,29.976071,-90.086517,1,Lounge,Convenience Store,Playground,Gas Station,Sandwich Place,Other Great Outdoors,Food Truck,Park,Women's Store,Food & Drink Shop
8,BLACK PEARL,29.935895,-90.134883,1,Food Truck,Plaza,Farmers Market,Scenic Lookout,Grocery Store,Donut Shop,Food Court,Food & Drink Shop,Food,Flower Shop
9,BROADMOOR,29.946568,-90.103812,1,Recreation Center,Food Truck,Coffee Shop,Taco Place,Music Venue,Ethiopian Restaurant,Frame Store,Food Court,Food & Drink Shop,Food
10,MARLYVILLE - FONTAINEBLEAU,29.953001,-90.113468,1,Bar,Nightclub,Food & Drink Shop,Cocktail Bar,Video Store,Lounge,Campground,Women's Store,Event Space,Food Truck


In [158]:
# create map

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(neworleans_merged['Latitude'], neworleans_merged['Longitude'], neworleans_merged['Neighborhood'], neworleans_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=1.0).add_to(map_clusters)
       
map_clusters

In [159]:
cluster0_raw = neworleans_merged.loc[neworleans_merged['Cluster Labels'] == 0, neworleans_merged.columns[[3] + list(range(5, neworleans_merged.shape[1]))]]
cluster0 = cluster0_raw.drop_duplicates()
cluster0

,Cluster Labels,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
66,0,Women's Store,Ethiopian Restaurant,Frame Store,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop,Fast Food Restaurant


In [160]:
cluster1_raw = neworleans_merged.loc[neworleans_merged['Cluster Labels'] == 1, neworleans_merged.columns[[3] + list(range(5, neworleans_merged.shape[1]))]]
cluster1 = cluster1_raw.drop_duplicates()
cluster1

,Cluster Labels,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,Ethiopian Restaurant,French Restaurant,Frame Store,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop,Fast Food Restaurant
1,1,Scenic Lookout,Historic Site,Boat or Ferry,Park,Gift Shop,Music Venue,Grocery Store,Coffee Shop,Bed & Breakfast
3,1,Park,College Arts Building,Coffee Shop,Plaza,Smoothie Shop,Sandwich Place,Outdoors & Recreation,Light Rail Station,Food
4,1,Snack Place,Playground,General Travel,Women's Store,Ethiopian Restaurant,Food Court,Food & Drink Shop,Food,Flower Shop
5,1,Breakfast Spot,Home Service,Gym / Fitness Center,Eastern European Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop
6,1,Convenience Store,Playground,Gas Station,Sandwich Place,Other Great Outdoors,Food Truck,Park,Women's Store,Food & Drink Shop
8,1,Plaza,Farmers Market,Scenic Lookout,Grocery Store,Donut Shop,Food Court,Food & Drink Shop,Food,Flower Shop
9,1,Food Truck,Coffee Shop,Taco Place,Music Venue,Ethiopian Restaurant,Frame Store,Food Court,Food & Drink Shop,Food
10,1,Nightclub,Food & Drink Shop,Cocktail Bar,Video Store,Lounge,Campground,Women's Store,Event Space,Food Truck
11,1,Seafood Restaurant,Gas Station,Asian Restaurant,Tennis Court,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop


In [161]:
cluster2_raw = neworleans_merged.loc[neworleans_merged['Cluster Labels'] == 2, neworleans_merged.columns[[3] + list(range(5, neworleans_merged.shape[1]))]]
cluster2 = cluster2_raw.drop_duplicates()
cluster2

,Cluster Labels,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,2,Park,Women's Store,Eastern European Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop,Fast Food Restaurant
48,2,Park,Harbor / Marina,Women's Store,Eastern European Restaurant,Food Truck,Food Court,Food & Drink Shop,Flower Shop,Fast Food Restaurant
55,2,Women's Store,Ethiopian Restaurant,Frame Store,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop,Fast Food Restaurant


In [162]:
cluster3_raw = neworleans_merged.loc[neworleans_merged['Cluster Labels'] == 2, neworleans_merged.columns[[3] + list(range(5, neworleans_merged.shape[1]))]]
cluster3 = cluster2_raw.drop_duplicates()
cluster3

,Cluster Labels,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,2,Park,Women's Store,Eastern European Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop,Fast Food Restaurant
48,2,Park,Harbor / Marina,Women's Store,Eastern European Restaurant,Food Truck,Food Court,Food & Drink Shop,Flower Shop,Fast Food Restaurant
55,2,Women's Store,Ethiopian Restaurant,Frame Store,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop,Fast Food Restaurant


In [167]:
neworleans_yoga = neworleans_grouped[['Neighborhood','Yoga Studio']]
neworleans_yoga

,Neighborhood,Yoga Studio
0,ALGIERS POINT,0.000000
1,AUDUBON,0.000000
2,B. W. COOPER,0.000000
3,BAYOU ST. JOHN,0.000000
4,BLACK PEARL,0.000000
5,BROADMOOR,0.000000
6,BYWATER,0.000000
7,CENTRAL BUSINESS DISTRICT,0.000000
8,CENTRAL CITY,0.000000
9,CITY PARK,0.000000
